# YOLO Head Detection – Counting Odaklı Metrik Analizi
Bu notebook, farklı CONF ve IoU ayarları altında elde edilen YOLO tabanlı head detection sayım sonuçlarını **tamamen counting perspektifiyle** analiz eder.

## 1. Problem Tanımı
Amaç: Görüntülerde **kaç adet kafa olduğunu doğru saymak**.

Bu nedenle bounding box kalitesinden ziyade **sayım doğruluğu** önceliklidir.

## 2. Kullanılan Metrikler ve Anlamları
| Metrik | Açıklama |
|---|---|
| MAE | Ortalama mutlak hata. Görüntü başına ortalama kaç kafa yanılıyoruz. |
| RMSE | Büyük hataları daha fazla cezalandıran hata metriği. |
| MAPE | Oransal hata. GT küçükken oynak olabilir. |
| GT_total | Tüm veri setindeki gerçek kafa sayısı. |
| Pred_total | Modelin tahmin ettiği toplam kafa sayısı. |
| Bias | Pred_total − GT_total. Sistematik fazla/eksik sayımı gösterir. |

## 3. CONF Sweep Sonuçları
| CONF | MAE | RMSE | MAPE | Pred_total | Bias |
|---|---|---|---|---|---|
| 0.25 | 1.918 | 5.708 | 0.0797 | 19466 | +861 |
| 0.35 | 1.676 | 4.571 | 0.0731 | 18876 | +271 |
| 0.45 | 1.779 | 4.083 | 0.0707 | 18232 | -373 |

### CONF Yorumları
- **0.25**: Çok fazla false positive → aşırı overcount
- **0.35**: En düşük MAE → counting için ideal bölge
- **0.45**: RMSE düşük ama undercount başlıyor

## 4. IoU Sweep (CONF=0.35)
| IoU | MAE | RMSE | Pred_total | Bias |
|---|---|---|---|---|
| 0.55 | 1.664 | 4.567 | 18864 | +259 |
| 0.50 | 1.667 | 4.523 | 18850 | +245 |
| 0.45 | 1.666 | 4.518 | 18841 | +236 |

### IoU Yorumları
IoU değişimi MAE üzerinde çok sınırlı etkiye sahip.
Bu, modelin bounding box kalitesinin zaten yeterli olduğunu gösterir.

## 5. En Dengeli Nokta
| Ayar | MAE | RMSE | Bias |
|---|---|---|---|
| conf40_iou50 | 1.70 | 4.33 | -74 |

Toplam sayım açısından **en dengeli** yapılandırma.

## 6. Nihai Sonuç
- **Counting odaklı production ayarı:** conf=0.40, iou=0.50
- **En düşük ortalama hata:** conf=0.35
- Conf/IoU artık ince ayar; esas kazanç post-process ve duplicate suppression tarafında.